# 绘制波形

在界面上设置日期表单，选择希望查询的起始时刻。
数据库可能保存很长时间的历史数据，
所以不可能全部返回，原因：
- 数据量太大，无法显示完全
- 网络带宽占用过大，系统无法快速显示

所以加一个算法，我们根据屏幕的宽度，
获取一屏能够显示的时间范围。
例如十秒，那么我们就在数据库查询一段时间范围内的波形。

## 数据表

波形是由16位数据构成的。MySQL 表示为：
- TINYINT：1字节
- SMALLINT：2字节
- MEDIUMINT：3字节
- INT：4字节
- BIGINT：8字节

增加时间字段，记录采样点发生的时间，便于程序按照时间范围来查询。

```python
class Sample:
    value = 2000
    time = '2020-07-10 12:11:10:001'
```

新建数据库表：
```sql
CREATE TABLE `medical_monitor`.`sample` (
  `value` SMALLINT NOT NULL,
  `time` TIMESTAMP NOT NULL);
```

数据库的时间戳类型，能够表示'1970-01-01 00:00:01'至'2038-01-19 03:14:07'之间精确到毫秒。
对于我们的系统足够使用。
该类型称为unix epoch时间戳，其实是转换而来，从1970年开始的秒数，表示成整数类型。

unix系统都采用这种方法表示文件的时间。
有一个著名的苹果时间戳bug，即手机设置为1970年之前的时间，
会导致时间变量越界，系统无法启动。

## 设备波形上传
```cpp
QSqlQuery query(db);

query.exec("INSERT INTO sample (value, time) "
           "VALUES (2000, '2020-07-10 12:11:10')");
```

另外一种方法，因为unix时间戳等同于整型，所以也可以用整数表示。

### 毫秒周期
本节试错环节，时间不允许可以跳过。

以ECG波形为例，每一秒产生500次波形。

所以我们需要以毫秒表示，因此增加一个字段，表示毫秒数。
```sql
ALTER TABLE `medical_monitor`.`sample` 
ADD COLUMN `ms` SMALLINT NULL;
```
修改表格的语法，增加列。

每次取到一个数据，都增加一行。
这样每秒钟会执行500次。
当我们有多道波形，或者多台设备时，数据库承受的写入数据量特别大。
这会极大影响性能。

### 多列
毫秒列的方法，每秒需要执行500个SQL语句，问题包括：
- 需要大量网络带宽。每条SQL均通过网络发送到数据库服务器执行，而网络的延迟会很高。
- 数据库写入操作的数量大。

所以我们需要对数据库的设计进行优化。
因为每两个采样是等周期的（采样频率恒定），
所以如果知道第一个采样的时刻，能够推导出第n个采样周期的时刻。

从数据结构的原理来看，我们可以把一秒的500个数据保存在一个数组内。
知道数组第一个采样的发生时间，其余采样的时间都能够根据周期推导。
同理，我们可以保存在1000个数据的数组。

为了减少SQL操作的次数，可以把整个数组一次性的保存到数据库。
这里的优化问题等效为如何在数据表的一行保存数组。解决方法：
- 500元的数组对应500列
- 数组串行化

串行化
```sql
ALTER TABLE `medical_monitor`.`sample` 
CHANGE COLUMN `value` `value` VARBINARY(1000) NOT NULL ;
```
因为数组的每个元素占2字节，500元数组一共1000字节，
所以把value字段改成1000字节的可变长度二进制类型。

插入数据的SQL变成了
```sql
INSERT INTO sample(time, value)
VALUES('2020-07-10 12:11:10', x'FA34E10293CB42848573A4E39937F479')
```
二进制是十六进制的字符串表示，即0xFA, 0x34, ...

接下来的问题转化成，Qt如何把数组变成SQL语句。
为了简化问题，我们只传三个数据的数组。
```cpp
query.prepare("INSERT INTO sample(time, value)"
              "VALUES(:time, :array)");

// 必须用QByteArray与数据库的二进制类型映射
QByteArray waves;
waves.resize(3*2); // 大小与数据数组一致
waves[0] = 2000 & 0xff;  // 以数据值2000为例
waves[1] = 2000 >> 8;

query.bindValue(":array", waves);

// 当前的时间
query.bindValue(":time", QDateTime::currentDateTime());
```

利用QByteArray与数据库的二进制类型进行相互映射。
第一种方式，修改QByteArray的大小与采样数组的大小一致，以字节为单位。
之后做一个循环，遍历的向QByteArray插入值。

```cpp
//三个数据为例
short samples[3] = {2000, 2100, 1900};
QByteArray waves2((char*)samples, 6);
query.bindValue(":array", waves2);
```
第二种方法，直接把数组复制生成新的QByteArray。
注意，构造函数的时候也需要指明数组的大小。

执行exec方法，运行SQL，将数组插入数据表。
之后用工具查看，最前方两个字节为：
0xD0 0x07，对应2字节整型为0x7D0，即十进制的2000.

## 波形显示
```cpp
query.prepare("SELECT * FROM sample "
              "WHERE time >= :start and time <= :end");
query.bindValue(":start", "2020-07-20 00:50:26");
query.bindValue(":end", "2020-07-20 00:50:56");

if(query.exec() && query.next())
{
    QByteArray waves = query.value("value").toByteArray();
    qDebug() << waves << waves.size();
}
```

按照时间范围取出数组，并且赋值到QByteArray。
再进行后续的波形显示工作。